## トービットモデルについて

トービットモデル（Tobit Model）は、経済学や他の社会科学で一般的に使用される回帰モデルで、特に左または右で検閲されたデータを扱うために設計されています。例えば、ある値以下または以上の値が検出不能な場合や、報告されない場合に適用されます。

### トービットモデルの基本概念

トービットモデルは、連続応答変数 $Y_i^*$ が観測できないか、一定の閾値（トランケーションポイント）により検閲される場合に使用されます。観測される応答変数 $Y_i$ は次のように定義されます：

$$
Y_i =
\begin{cases} 
Y_i^* & \text{if } Y_i^* > L \\
L & \text{if } Y_i^* \leq L
\end{cases}
$$

ここで、
- $Y_i^*$ は潜在変数（観測されない真の値）
- $L$ は検閲の閾値（例えば、$L = 0$ の場合、非負のデータが観測される）

潜在変数 $Y_i^*$ は次のように定義されます：

$$
Y_i^* = X_i \beta + \epsilon_i
$$

ここで、
- $X_i$ は説明変数のベクトル
- $\beta$ はパラメータのベクトル
- $\epsilon_i$ は通常の誤差項で、標準正規分布に従う

### トービットモデルの利点と制限
- **利点**:
  - データの検閲を扱うことができるため、切り捨てられたデータを含む分析に適している。
  - OLS回帰と比較して、検閲データの影響を考慮することができる。
- **制限**:
  - モデルの仮定が正しくない場合、結果がバイアスされる可能性がある。
  - 分布の仮定（例えば、誤差項が正規分布に従うこと）が厳格である。

### 実例

以下にPythonを使って、トービットモデルを適用する例を示します。ここでは、`statsmodels`ライブラリを使用します。

### コードの説明
1. **データの作成**: 説明変数としてランダムなデータを生成し、検閲された応答変数を生成します。潜在変数 $y^*$ は観測されず、$y$ は検閲されたデータです。
2. **TobitModelクラスの定義**: `GenericLikelihoodModel`を拡張して、トービットモデルの対数尤度関数を実装します。
3. **トービットモデルのフィッティング**: データを用いてトービットモデルをフィッティングします。
4. **結果の表示**: モデルの結果を表示します。
5. **予測値の計算とプロット**: フィッティングされたモデルを使って予測値を計算し、データと共にプロットします。

このカスタムトービットモデルにより、`statsmodels`を使用して検閲されたデータを分析することができます。

In [10]:
import numpy as np
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from statsmodels.base.model import GenericLikelihoodModel

# サンプルデータの作成
np.random.seed(42)
n = 100
X = np.random.rand(n) * 10
X = sm.add_constant(X)
beta = [1, 0.5]
sigma = 1
y_star = np.dot(X, beta) + np.random.normal(0, sigma, n)
L = 0
y = np.maximum(y_star, L)

# データフレームの作成
data = pd.DataFrame({'y': y, 'const': X[:, 0], 'x1': X[:, 1]})

class TobitModel(GenericLikelihoodModel):
    def __init__(self, endog, exog, left=0, *args, **kwargs):
        self.left = left
        super(TobitModel, self).__init__(endog, exog, *args, **kwargs)

    def nloglikeobs(self, params):
        exog = self.exog
        endog = self.endog
        left = self.left
        beta = params[:-1]
        sigma = params[-1]
        xb = np.dot(exog, beta)
        llf = np.where(endog > left,
                       np.log(norm.pdf((endog - xb) / sigma)) - np.log(sigma),
                       np.log(norm.cdf((left - xb) / sigma)))
        return -llf

    def fit(self, start_params=None, maxiter=10000, maxfun=5000, **kwds):
        if start_params is None:
            start_params = np.append(np.zeros(self.exog.shape[1]), 1)
        return super(TobitModel, self).fit(start_params=start_params,
                                           maxiter=maxiter, maxfun=maxfun, **kwds)

# トービットモデルのフィッティング
tobit_model = TobitModel(data['y'], data[['const', 'x1']], left=L)
tobit_result = tobit_model.fit()

# 結果の表示
print(tobit_result.summary())

# 予測値の計算
y_pred = tobit_result.predict(data[['const', 'x1']])

# プロット
plt.scatter(data['x1'], data['y'], label="Data")
plt.plot(data['x1'], y_pred, color='red', label="Tobit model")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.show()


Optimization terminated successfully.
         Current function value: 1.311465
         Iterations: 191
         Function evaluations: 347
                              TobitModel Results                              
Dep. Variable:                      y   Log-Likelihood:                -131.15
Model:                     TobitModel   AIC:                             268.3
Method:            Maximum Likelihood   BIC:                             276.1
Date:                Sat, 06 Jul 2024                                         
Time:                        08:33:31                                         
No. Observations:                 100                                         
Df Residuals:                      98                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

/Users/taketoaramaki/study/statistics_study/.venv/lib/python3.12/site-packages/statsmodels/base/model.py:2748: UserWarning: df_model + k_constant + k_extra differs from k_params
  warnings.warn("df_model + k_constant + k_extra "
/Users/taketoaramaki/study/statistics_study/.venv/lib/python3.12/site-packages/statsmodels/base/model.py:2752: UserWarning: df_resid differs from nobs - k_params
  warnings.warn("df_resid differs from nobs - k_params")


NotImplementedError: 